In [1]:
import os

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

load_dotenv()

# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatOpenAI(model=os.getenv("DEFAULT_MODEL"))

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='저녁 먹자구요? 싫어요. 아버지를 찾아야 해서요. 아버지를 찾아야 해서 시간 내기가 어려울 것 같아요. 게다가 저는 이런 시골에서 혼자 식사하는 게 익숙하지 않아요. 마을 사람들과 함께 식사하는 것을 더 좋아해요. 죄송해요, 다른 때 뵐 수 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 60, 'total_tokens': 139, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'accounts/fireworks/models/llama4-maverick-instruct-basic', 'system_fingerprint': None, 'id': '7105b825-e4a5-4072-a122-cdf41a5829dd', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--667ae2e1-190b-4005-9c29-f29300307067-0', usage_metadata={'input_tokens': 60, 'output_tokens': 79, 'total_tokens': 139, 'input_token_details': {}, 'output_token_details': {}})

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'안녕하십니까, 개스톤 씨. 저는 벨입니다. 저녁에요? 저는 아버지를 찾아야 해서요. 아버지가 실종되었는데, 제가 찾아야 한다고요. 죄송하지만 오늘은 약속이 있어서요.'

In [3]:
chain = model | parser
chain.invoke(messages)

'안녕, 개스톤. 하지만 오늘 저녁은 사양할래. 아버지가 오실 거라고 했거든. 아버지를 기다리다가 늦어지면 내가 먹을 수 있게 저녁을 준비해야 하니까. 게다가 나는 아직 마법사의 저주에 걸린 성에서 빠져나올 방법을 찾고 있어. 네가 도와줄 수 있어?'

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [5]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'저녁은 함께하지 못할 것 같아요. 제 아버지가 실종되어서 그를 찾아야 해요. 사실 그분이 위험한 상황에 있는 것 같아요. 아버지를 찾아야겠어요. 죄송해요, 야수.'

In [8]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'저녁을요? 전 오늘 집안일 때문에 바빠서 저녁 먹을 시간도 없을 것 같아요. 그리고 전 요즘 책 읽는 걸 좋아해서, 조용한 곳에서 책 읽는 시간을 보내는 것을 더 좋아해요. 죄송하지만 오늘은 혼자 있고 싶어요.'

In [7]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})


Adlib(answer='미안하지만, 오늘 저녁은 곤란해요. 전 독서에 더 관심이 있고, 마을 사람들과 어울리는 것보다 조용히 책을 읽는 걸 좋아하거든요. 게다가, 마을에서 당신과 같이... 거친 사람과 식사하는 건 좀 부담스러워요.', main_emotion='불쾌', main_emotion_intensity=7.0)

##### model.with_structured_output(Adlib) - request, response body
```text
REQUEST >> https://api.fireworks.ai/inference/v1/chat/completions
{
  "messages": [
    {
      "content": "너는 미녀와 야수에 나오는 벨 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.",
      "role": "system"
    },
    {
      "content": "안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?",
      "role": "user"
    }
  ],
  "model": "accounts/fireworks/models/llama4-maverick-instruct-basic",
  "response_format": {
    "type": "json_schema",
    "json_schema": {
      "schema": {
        "description": "스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스",
        "properties": {
          "answer": {
            "description": "스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사",
            "title": "Answer",
            "type": "string"
          },
          "main_emotion": {
            "description": "대사의 주요 감정",
            "enum": [
              "기쁨",
              "분노",
              "슬픔",
              "공포",
              "냉소",
              "불쾌",
              "중립"
            ],
            "title": "Main Emotion",
            "type": "string"
          },
          "main_emotion_intensity": {
            "description": "대사의 주요 감정의 강도 (0.0 ~ 1.0)",
            "title": "Main Emotion Intensity",
            "type": "number"
          }
        },
        "required": [
          "answer",
          "main_emotion",
          "main_emotion_intensity"
        ],
        "title": "Adlib",
        "type": "object",
        "additionalProperties": false
      },
      "name": "Adlib",
      "strict": true
    }
  },
  "stream": false
}
RESPONSE >> 200 OK
{
  "id": "7d61c7a5-abaa-47d0-aae5-7a7a56f24187",
  "object": "chat.completion",
  "created": 1754890062,
  "model": "accounts/fireworks/models/llama4-maverick-instruct-basic",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n\"answer\": \"미안하지만, 오늘 저녁은 곤란해요. 전 독서에 더 관심이 있고, 마을 사람들과 어울리는 것보다 조용히 책을 읽는 걸 좋아하거든요. 게다가, 마을에서 당신과 같이... 거친 사람과 식사하는 건 좀 부담스러워요.\"\n,\n\"main_emotion\": \"불쾌\",\n\"main_emotion_intensity\": 7\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 59,
    "total_tokens": 153,
    "completion_tokens": 94
  }
}
```